In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat  # this is the SciPy module that loads mat-files

mat = loadmat('data_raw/participant6_0.mat')
df = pd.DataFrame(mat['data'])

print(df[1])
df.info()

0         530815
1         531219
2         531508
3         531832
4         642831
           ...  
213969    890773
213970    898805
213971    906790
213972    907193
213973    922780
Name: 1, Length: 213974, dtype: int32
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213974 entries, 0 to 213973
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   0       213974 non-null  int32
 1   1       213974 non-null  int32
 2   2       213974 non-null  int32
 3   3       213974 non-null  int32
 4   4       213974 non-null  int32
dtypes: int32(5)
memory usage: 4.1 MB
